## HLoc for Tramway Dataset


### Step 1: Reconstruct a full dataset

cp -r /media/siyanhu/T71/DJI/20250124/DJI_20250124123335_0013_D_B01/images /media/siyanhu/T71/lidarcam/rgb7

cp -r /media/siyanhu/T71/DJI/20250124/DJI_20250124123336_0014_D_C01/images /media/siyanhu/T71/lidarcam/rgb8

cp -r /media/siyanhu/T71/DJI/20250124/DJI_20250124123337_0013_D/images /media/siyanhu/T71/lidarcam/rgb9

In [2]:
import os
import struct
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm
from pathlib import Path
import numpy as np
import collections
from hloc import extract_features, match_features, reconstruction,pairs_from_retrieval

import torch
import torchvision.transforms as transforms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
# for i in range (0, 6):
#     image_folder = "/media/siyanhu/T71/hloc/datasets/rgb" + str(i + 1)
#     image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]
#     image_files.sort(key=lambda x: float(x.split(".")[0]))

#     for idx, filename in enumerate(image_files, start=1):
#         new_name = f"{idx:05d}.jpg"
#         old_path = os.path.join(image_folder, filename)
#         new_path = os.path.join(image_folder, new_name)
#         os.rename(old_path, new_path)
# print("Renaming complete!")

In [5]:
retrieval_conf = extract_features.confs['netvlad']
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superpoint+lightglue']

# Force GPU usage
retrieval_conf['device'] = 'cuda'  # Ensure NetVLAD uses GPU
feature_conf['device'] = 'cuda'    # Ensure SuperPoint uses GPU
matcher_conf['device'] = 'cuda'    # Ensure LightGlue uses GPU

In [7]:
# Paths
images = Path('/media/siyanhu/T71/hloc/datasets')
outputs = Path('/media/siyanhu/T71/hloc/dji_recon')

sfm_pairs = outputs / 'pairs-netvlad.txt'
loc_pairs = outputs / 'pairs-loc.txt'

sfm_dir = outputs / 'sfm'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
retrieval_features = outputs/'features_retrieval.h5'

In [8]:
# ref_seqs = [1, 2, 3, 4, 5, 6]
# query_seqs = [7, 8, 9]

ref_seqs = [7, 8, 9]

# full_paths = []
ref_paths = []
# query_paths = []

for i in range(9):
    seq_num = i + 1
    if seq_num in ref_seqs:
        for p in (images / f'rgb{seq_num}/').iterdir():
            if '.jpg' in str(p.relative_to(images)):
                    ref_paths.append(str(p.relative_to(images)))
    #                 full_paths.append(str(p.relative_to(images)))
    # elif seq_num in query_seqs:
    #     for p in (images / f'rgb{seq_num}/').iterdir():
    #         if '.jpg' in str(p.relative_to(images)):
    #                 query_paths.append(str(p.relative_to(images)))
    #                 full_paths.append(str(p.relative_to(images)))
    
print(ref_paths[5:10], ref_paths[-10:-5])
# print(query_paths[5:10], query_paths[-10:-5])
# print(full_paths[5:10], full_paths[-10:-5])

['rgb7/00006.jpg', 'rgb7/00007.jpg', 'rgb7/00008.jpg', 'rgb7/00009.jpg', 'rgb7/00010.jpg'] ['rgb9/00747.jpg', 'rgb9/00748.jpg', 'rgb9/00749.jpg', 'rgb9/00750.jpg', 'rgb9/00751.jpg']


In [9]:
retrieval_path = extract_features.main(retrieval_conf, images, image_list = ref_paths, feature_path = retrieval_features)
print(retrieval_path)
pairs_from_retrieval.main(retrieval_path, sfm_pairs, num_matched=20)

[2025/03/17 11:58:45 hloc INFO] Extracting local features with configuration:
{'device': 'cuda',
 'model': {'name': 'netvlad'},
 'output': 'global-feats-netvlad',
 'preprocessing': {'resize_max': 1024}}
[2025/03/17 11:58:45 hloc INFO] Skipping the extraction.
[2025/03/17 11:58:45 hloc INFO] Extracting image pairs from a retrieval database.


/media/siyanhu/T71/hloc/dji_recon/features_retrieval.h5


[2025/03/17 11:58:47 hloc INFO] Found 45360 pairs.


In [10]:
extract_features.main(feature_conf, images, image_list=ref_paths, feature_path=features)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches)

[2025/03/17 11:58:48 hloc INFO] Extracting local features with configuration:
{'device': 'cuda',
 'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
[2025/03/17 11:58:48 hloc INFO] Skipping the extraction.
[2025/03/17 11:58:48 hloc INFO] Matching local features with configuration:
{'device': 'cuda',
 'model': {'features': 'superpoint', 'name': 'lightglue'},
 'output': 'matches-superpoint-lightglue'}
[2025/03/17 11:58:49 hloc INFO] Skipping the matching.


PosixPath('/media/siyanhu/T71/hloc/dji_recon/matches.h5')

In [11]:
model = reconstruction.main(sfm_dir, images, sfm_pairs, features, matches,image_list = ref_paths)

[2025/03/17 11:58:51 hloc WARNING] The database already exists, deleting it.
[2025/03/17 11:58:51 hloc INFO] Creating an empty database...
[2025/03/17 11:58:51 hloc INFO] Importing images into the database...
[2025/03/17 12:00:41 hloc INFO] Importing features into the database...


  0%|          | 0/2268 [00:00<?, ?it/s]

[2025/03/17 12:00:46 hloc INFO] Importing matches into the database...


  0%|          | 0/45360 [00:00<?, ?it/s]

[2025/03/17 12:01:23 hloc INFO] Performing geometric verification of the matches...
I20250317 12:01:23.609635 131836772615872 misc.cc:44] 
Feature matching
I20250317 12:01:23.612657 131835898103488 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612771 131835407369920 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612822 131836762130112 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612862 131835646445248 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612868 131835877131968 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612922 131835887617728 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.612975 131835396884160 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.613130 131835375912640 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.613152 131835227014848 sift.cc:1432] Creating SIFT CPU feature matcher
I20250317 12:01:23.613188 13183538